# Test of the GHCN Class

In [1]:
import os
import pandas as pd
import datetime
from ghcn_monthly_feed import GHCN

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
G = GHCN(downloadDir='/home/dmasson/data/GHCNM/',
         logfilename='/home/dmasson/temp/ghcnm01.log')
self = G

In [ ]:
# Get all data (metadata and observations)

# G.wgetData()

In [5]:
# Create the station collection (only needed once)

# self.upsertStationCollection()

In [4]:
# Create indexes

# self.createStationIndexing()

In [ ]:
country_df = pd.read_fwf('ghcnm-countries.txt',
                     colspecs=[[0,2], [3, 500]],
                     header=None,
                     names=['country_id', 'country'])
country_df.head()

In [ ]:
path = self.findNewestFile(pattern='.inv')
path

In [ ]:
# Station metadata
sta_df = pd.read_fwf(path,
                     colspecs=[[0,11], [0,2],[2,8],[13, 20], [24, 30], [31,37], [38,69], [90, 106],[106,107]],
                     header=None,
                     names=['station_id','country_id','wmo_id', 
                            'lat', 'lon', 'elev', 'name', 'landcover', 'popclass'])
sta_df = pd.merge(sta_df, country_df, on='country_id')
sta_df.head()

In [ ]:
sta_df.head()

In [ ]:
col_sta = self._createMongoConn()['col_sta']

In [ ]:
i = 0 
newdoc = dict({'station_id': str(sta_df.station_id[i]), 
                   'loc': {'type': 'Point', 'coordinates': [float(sta_df.lon[i]), float(sta_df.lat[i])]}, 
                   'country': sta_df.country[i], 
                   'country_id': str(sta_df.country_id[i]), 
                   'wmo_id': str(sta_df.wmo_id[i]), 
                   'elev': sta_df.elev[i], 
                   'name': sta_df.name[i], 
                   'landcover': sta_df.landcover[i], 'popclass': sta_df.popclass[i]})
col_sta.update_one(filter={"station_id": newdoc['station_id']}, 
                   update=dict({'$set': newdoc}), upsert=True)